In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
!pip install gcsfs

In [3]:
# SIMPLIFIED EFFICIENT ZARR SAVING
# Focus: Proper parallelism, simple and reliable
# Removed complex auto-detection that might cause issues

import os
import time
import xarray as xr
from numcodecs import Blosc
import gcsfs

# Re-use a global filesystem client when possible
gcs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')

def save_dataset_efficient_zarr(
    ds,
    zarr_path,
    chunk_sizes=None,
    compression='lz4',
    compression_level=1,
    overwrite=True,
    consolidated=True,
    storage='auto',
    gcs_project=None,
):
    """
    Simplified zarr saving – focuses on reliable parallelism.

    Parameters
    ----------
    ds : xarray.Dataset
        Dataset to save (lazy dask arrays or in-memory).
    zarr_path : str
        Destination path or GCS URI (e.g. gs://bucket/path.zarr).
    chunk_sizes : dict, optional
        Chunk sizes per dimension (e.g. {'time': 20, 'x': 256, 'y': 256}).
    compression : {'lz4','blosc','zstd',None} or dict
        Built-in compressor choice or explicit encoding dict.
    compression_level : int
        Compression level (1 fastest, 9 best compression).
    overwrite : bool
        Overwrite existing zarr store.
    consolidated : bool
        Create consolidated metadata (recommended).
    storage : {'auto','local','gcs'}
        Force storage backend or infer from path when 'auto'.
    gcs_project : str, optional
        Explicit GCP project for a fresh filesystem client.

    Returns
    -------
    str
        The zarr_path that was written.
    """
    def _format_size(num_bytes: int) -> str:
        size_mb = num_bytes / (1024 * 1024)
        size_gb = size_mb / 1024
        return f"{size_gb:.2f} GB" if size_gb >= 1 else f"{size_mb:.2f} MB"

    start_time = time.time()

    storage = storage.lower()
    if storage == 'auto':
        storage = 'gcs' if zarr_path.startswith('gs://') else 'local'
    if storage not in {'local', 'gcs'}:
        raise ValueError("storage must be one of {'auto', 'local', 'gcs'}")

    fs = None
    if storage == 'gcs':
        fs = gcs if gcs_project is None else gcsfs.GCSFileSystem(project=gcs_project)
    else:
        zarr_dir = os.path.dirname(zarr_path) if os.path.dirname(zarr_path) else '.'
        if zarr_dir and not os.path.exists(zarr_dir):
            os.makedirs(zarr_dir, exist_ok=True)

    # Handle overwrite
    if storage == 'gcs':
        if fs.exists(zarr_path):
            if not overwrite:
                raise FileExistsError(
                    f"Zarr store already exists on GCS: {zarr_path}\n"
                    "Set overwrite=True to replace it."
                )
            print(f"🗑️  Removing existing GCS zarr store: {zarr_path}")
            fs.rm(zarr_path, recursive=True)
    else:
        if os.path.exists(zarr_path):
            if not overwrite:
                raise FileExistsError(
                    f"Zarr store already exists: {zarr_path}\n"
                    "Set overwrite=True to replace it."
                )
            import shutil
            print(f"🗑️  Removing existing zarr store: {zarr_path}")
            shutil.rmtree(zarr_path)

    # Default chunk sizes
    if chunk_sizes is None:
        chunk_sizes = {}
        dims = ds.dims
        if 'time' in dims:
            chunk_sizes['time'] = min(20, dims['time'])
        if 'x' in dims:
            chunk_sizes['x'] = min(256, dims['x'])
        if 'y' in dims:
            chunk_sizes['y'] = min(256, dims['y'])
        for dim_name, dim_len in dims.items():
            chunk_sizes.setdefault(dim_name, min(100, dim_len))

    print(f"📦 Saving to zarr: {zarr_path}")
    print(f"   Dimensions: {dict(ds.dims)}")
    print(f"   Chunks: {chunk_sizes}")
    print(f"   Compression: {compression} (level {compression_level})")
    print(f"   Storage: {storage}")

    # Prepare compression
    if compression == 'lz4':
        compressor = Blosc(cname='lz4', clevel=compression_level, shuffle=Blosc.SHUFFLE, blocksize=0)
        encoding = {var: {'compressor': compressor} for var in ds.data_vars}
    elif compression == 'blosc':
        compressor = Blosc(cname='blosclz', clevel=compression_level, shuffle=Blosc.SHUFFLE, blocksize=0)
        encoding = {var: {'compressor': compressor} for var in ds.data_vars}
    elif compression == 'zstd':
        compressor = Blosc(cname='zstd', clevel=compression_level, shuffle=Blosc.SHUFFLE, blocksize=0)
        encoding = {var: {'compressor': compressor} for var in ds.data_vars}
    elif compression is None:
        encoding = {}
    else:
        encoding = compression  # assume dict supplied

    # Chunk and save
    ds_chunked = ds.chunk(chunk_sizes)
    print("💾 Writing to zarr (with automatic parallelism)...")

    store = fs.get_mapper(zarr_path) if storage == 'gcs' else zarr_path
    try:
        from dask.diagnostics import ProgressBar
        with ProgressBar():
            ds_chunked.to_zarr(
                store,
                mode='w',
                encoding=encoding,
                consolidated=consolidated,
                compute=True,
            )
    except ImportError:
        ds_chunked.to_zarr(
            store,
            mode='w',
            encoding=encoding,
            consolidated=consolidated,
            compute=True,
        )

    elapsed = time.time() - start_time

    # Size reporting
    total_size = None
    if storage == 'gcs':
        try:
            size_info = fs.du(zarr_path)
            if isinstance(size_info, dict):
                total_size = sum(size_info.values())
            elif isinstance(size_info, (int, float)):
                total_size = size_info
        except Exception as exc:
            print(f"⚠️  Could not compute GCS store size: {exc}")
    else:
        if os.path.exists(zarr_path):
            total_size = 0
            for dirpath, _, filenames in os.walk(zarr_path):
                for f in filenames:
                    fp = os.path.join(dirpath, f)
                    total_size += os.path.getsize(fp)

    if total_size is not None:
        size_str = _format_size(total_size)
        write_speed = total_size / elapsed / (1024 * 1024)
        print("✅ Dataset saved successfully!")
        print(f"   Store size: {size_str}")
        print(f"   Time: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
        print(f"   Write speed: {write_speed:.1f} MB/s")
        print(f"   Path: {zarr_path}")
    else:
        print("✅ Dataset saved successfully! (size unavailable)")
        print(f"   Time: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
        print(f"   Path: {zarr_path}")

    return zarr_path


def load_dataset_zarr(zarr_path, consolidated=True, storage='auto', gcs_project=None):
    """
    Load a dataset from a zarr store located locally or on GCS.
    """
    storage = storage.lower()
    if storage == 'auto':
        storage = 'gcs' if zarr_path.startswith('gs://') else 'local'
    if storage not in {'local', 'gcs'}:
        raise ValueError("storage must be one of {'auto', 'local', 'gcs'}")

    if storage == 'gcs':
        fs = gcs if gcs_project is None else gcsfs.GCSFileSystem(project=gcs_project)
        if not fs.exists(zarr_path):
            raise FileNotFoundError(f"Zarr store not found on GCS: {zarr_path}")
        mapper = fs.get_mapper(zarr_path)
        print(f"📂 Loading dataset from GCS zarr: {zarr_path}")
        ds = xr.open_zarr(mapper, consolidated=consolidated)
    else:
        if not os.path.exists(zarr_path):
            raise FileNotFoundError(f"Zarr store not found: {zarr_path}")
        print(f"📂 Loading dataset from zarr: {zarr_path}")
        ds = xr.open_zarr(zarr_path, consolidated=consolidated)

    print(f"✅ Dataset loaded: {dict(ds.dims)}")
    return ds


print("✅ Simplified zarr saving functions loaded!")
print("\nKey simplifications:")
print("  - No complex auto-detection")
print("  - Always uses compute=True (let dask handle parallelism)")
print("  - Simple, reliable, focuses on parallelism")
print("  - Works with both lazy and in-memory arrays")


✅ Simplified zarr saving functions loaded!

Key simplifications:
  - No complex auto-detection
  - Always uses compute=True (let dask handle parallelism)
  - Simple, reliable, focuses on parallelism
  - Works with both lazy and in-memory arrays


In [4]:
# zarr_path = os.getenv('GCS_ZARR_DIR') + '/ds_resampled.zarr'
zarr_path = 'data/ds_resampled.zarr'
storage = 'local'

ds_resampled = load_dataset_zarr(zarr_path, storage=storage)
ds_resampled


📂 Loading dataset from zarr: data/ds_resampled.zarr
✅ Dataset loaded: {'time': 81, 'x': 4489, 'y': 3213}


/tmp/ipykernel_601/411130122.py:214: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


<xarray.Dataset> Size: 9GB
Dimensions:   (time: 81, x: 4489, y: 3213)
Coordinates:
    image_id  (time) <U17 6kB dask.array<chunksize=(20,), meta=np.ndarray>
  * time      (time) datetime64[ns] 648B 2018-02-15 2018-05-15 ... 2025-08-15
  * x         (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y         (y) float64 26kB 9.949e+06 9.949e+06 ... 9.982e+06 9.982e+06
Data variables:
    EVI       (time, x, y) float32 5GB dask.array<chunksize=(20, 256, 256), meta=np.ndarray>
    NDVI      (time, x, y) float32 5GB dask.array<chunksize=(20, 256, 256), meta=np.ndarray>
Attributes:
    crs:      EPSG:32749

In [5]:
print(ds_resampled)

<xarray.Dataset> Size: 9GB
Dimensions:   (time: 81, x: 4489, y: 3213)
Coordinates:
    image_id  (time) <U17 6kB dask.array<chunksize=(20,), meta=np.ndarray>
  * time      (time) datetime64[ns] 648B 2018-02-15 2018-05-15 ... 2025-08-15
  * x         (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y         (y) float64 26kB 9.949e+06 9.949e+06 ... 9.982e+06 9.982e+06
Data variables:
    EVI       (time, x, y) float32 5GB dask.array<chunksize=(20, 256, 256), meta=np.ndarray>
    NDVI      (time, x, y) float32 5GB dask.array<chunksize=(20, 256, 256), meta=np.ndarray>
Attributes:
    crs:      EPSG:32749


In [6]:
crs_ds = ds_resampled.attrs.get('crs')

In [ ]:
#### DS TRAIN CHECK RESAMPLING
import os
import geopandas as gpd
import pandas as pd

path_training = '00_input/training_shp/'

layers = [shp for shp in os.listdir(path_training) if shp.endswith('.shp') and shp.startswith('sample')]

gdf_list = []

for lyr in layers:
    print(lyr)
    gdf = gpd.read_file(os.path.join(path_training,lyr))
    gdf['layer'] = lyr.replace('.shp', '')
    print(gdf.crs)
    gdf_utm = gdf.to_crs(crs_ds)
    gdf_list.append(gdf_utm)

sample_2.shp
EPSG:4326
sample_3.shp
EPSG:4326
sample_1.shp
EPSG:32749


In [12]:
training_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
# training_gdf = gpd.read_file('00_input/training_shp/sample_1.shp')
# training_gdf.head()
training_gdf.head()

,t_202508,t_202507,t_202506,t_202505,t_202504,t_202503,t_202502,t_202501,t_202412,t_202411,...,t_202306_D,t_202309_D,t_202312_D,t_202403_D,t_202406_D,t_202409_D,t_202412_D,t_202503_D,t_202506_D,t_202509_D
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
training_gdf.crs

<Projected CRS: EPSG:32749>
Name: WGS 84 / UTM zone 49S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 108°E and 114°E, southern hemisphere between 80°S and equator, onshore and offshore. Australia. Indonesia.
- bounds: (108.0, -80.0, 114.0, 0.0)
Coordinate Operation:
- name: UTM zone 49S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
training_gdf.geometry.head()

0    POLYGON ((591831.671 9951866.774, 592488.463 9...
1    POLYGON ((590668.414 9951528.793, 590655.818 9...
2    POLYGON ((592279.476 9951073.580, 592242.830 9...
3    POLYGON ((591967.853 9951121.158, 591963.540 9...
4    POLYGON ((590410.775 9951324.006, 590396.290 9...
Name: geometry, dtype: geometry

In [15]:
# for i in training_gdf.columns:
#     print(i)

In [16]:
# training_gdf.plot()

In [17]:
# training_gdf.head()

In [18]:
training_gdf.columns

Index(['t_202508', 't_202507', 't_202506', 't_202505', 't_202504', 't_202503',
       't_202502', 't_202501', 't_202412', 't_202411',
       ...
       't_202306_D', 't_202309_D', 't_202312_D', 't_202403_D', 't_202406_D',
       't_202409_D', 't_202412_D', 't_202503_D', 't_202506_D', 't_202509_D'],
      dtype='object', length=109)

In [19]:
list_columns_time = [i for i in training_gdf.columns if i.startswith('t_') and not i.endswith('D')]
list_columns_time = list(sorted(list_columns_time))
list_columns_time

['t_201603',
 't_201606',
 't_201609',
 't_201612',
 't_201703',
 't_201706',
 't_201709',
 't_201712',
 't_201803',
 't_201806',
 't_201809',
 't_201812',
 't_201903',
 't_201906',
 't_201909',
 't_201912',
 't_202003',
 't_202006',
 't_202009',
 't_202012',
 't_202103',
 't_202106',
 't_202109',
 't_202112',
 't_202201',
 't_202202',
 't_202203',
 't_202204',
 't_202205',
 't_202206',
 't_202207',
 't_202208',
 't_202209',
 't_202210',
 't_202211',
 't_202212',
 't_202301',
 't_202302',
 't_202303',
 't_202304',
 't_202305',
 't_202306',
 't_202307',
 't_202308',
 't_202309',
 't_202310',
 't_202311',
 't_202312',
 't_202401',
 't_202402',
 't_202403',
 't_202404',
 't_202405',
 't_202406',
 't_202407',
 't_202408',
 't_202409',
 't_202410',
 't_202411',
 't_202412',
 't_202501',
 't_202502',
 't_202503',
 't_202504',
 't_202505',
 't_202506',
 't_202507',
 't_202508',
 't_202509']

In [20]:
columns_filter = ['layer'] + list_columns_time + ['geometry']

In [21]:
training_gdf_col_filtered = training_gdf[columns_filter]
training_gdf_col_filtered.head()
# training_gdf_col_filtered.plot()

,layer,t_201603,t_201606,t_201609,t_201612,t_201703,t_201706,t_201709,t_201712,t_201803,...,t_202501,t_202502,t_202503,t_202504,t_202505,t_202506,t_202507,t_202508,t_202509,geometry
0,sample_2,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,"POLYGON ((591831.671 9951866.774, 592488.463 9..."
1,sample_2,NaN,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,"POLYGON ((590668.414 9951528.793, 590655.818 9..."
2,sample_2,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,"POLYGON ((592279.476 9951073.580, 592242.830 9..."
3,sample_2,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,"POLYGON ((591967.853 9951121.158, 591963.540 9..."
4,sample_2,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,"POLYGON ((590410.775 9951324.006, 590396.290 9..."


In [ ]:
### TAKES TIME FOR THIS GEE since we will upload data to GEE
# import os
# import ee
# import geemap.foliumap as geemap

# service_account = os.getenv('SERVICE_ACCOUNT')
# key_path = '/usr/src/app/user_id.json'

# credentials = ee.ServiceAccountCredentials(service_account, key_path)
# ee.Initialize(credentials)

# gdf_wgs84 = training_gdf_col_filtered.to_crs(epsg=4326)

# centroid = gdf_wgs84.geometry.centroid.unary_union.centroid
# m = geemap.Map(center=[centroid.y, centroid.x], zoom=9, ee_initialize=False)
# m.add_gdf(gdf_wgs84, layer_name="training_gdf_col_filtered")
# m

In [24]:
# training_gdf_col_filtered.to_file("00_input/training_shp/training_gdf_col_filtered_4326.shp")

In [25]:
### 

In [26]:
## lets load in wmts from gdf
gdf_wgs84 = training_gdf_col_filtered.to_crs(epsg=4326)

#### wmts loading
from wfs_manager import WFSManager
import geemap

fc = geemap.geopandas_to_ee(gdf_wgs84)

wfs = WFSManager(fastapi_url="http://fastapi:8000", wfs_base_url="http://localhost:8001")
wfs.addLayer(fc, "Training data collected")
wfs.publish()

/opt/venv/lib/python3.12/site-packages/geemap/conversion.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Could not convert the geojson to ee.Geometry()


Exception: Earth Engine client library not initialized. See http://goo.gle/ee-auth.

In [ ]:
import pandas as pd

# Get all columns that start with 't_'
t_cols = ['t_202509', 't_202506', 't_202503', 't_202412', 't_202409', 't_202406',
       't_202403', 't_202312', 't_202309', 't_202306', 't_202303', 't_202212',
       't_202209', 't_202206', 't_202203', 't_202112', 't_202109', 't_202106',
       't_202103', 't_202012', 't_202009', 't_202006', 't_202003', 't_201912',
       't_201909', 't_201906', 't_201903', 't_201812', 't_201809', 't_201806',
       't_201803', 't_201712', 't_201709', 't_201706', 't_201703', 't_201612']

# Melt the dataframe with geometry as id_var
df_long = pd.melt(
    training_gdf, 
    id_vars=['geometry'],
    value_vars=t_cols,
    var_name='time_period',
    value_name='value'
)

# Set geometry as index
df_long = df_long.set_index('geometry')

print(f"Original shape: {training_gdf.shape}")
print(f"Long format shape: {df_long.shape}")
print(f"Time columns found: {len(t_cols)}")
print(f"\nFirst few rows:")
df_long.head()


In [ ]:
df_long.reset_index(inplace=True)
df_long.head()
df_long.plot()


In [ ]:
df_long

In [ ]:
test = 't_202509'
test[2:]


In [ ]:
df_long = df_long.rename(columns={'value': 'type'})
df_long['date'] = df_long['time_period'].str[2:].astype(int)
df_long.head()


In [ ]:
type(df_long)

In [ ]:
training_gdf = df_long.copy()
# Remove multipolygons
size = len(training_gdf)
training_gdf = training_gdf.explode()

In [ ]:
try:
    assert size == len(training_gdf)
except AssertionError:
    print(f'Exploded training_gdf failed')

In [ ]:
#####
training_gdf['date'] = pd.to_datetime(training_gdf['date'], format='%Y%m')

# add year columnt
training_gdf['year'] = training_gdf['date'].dt.year

# convert column 'type' from string to int
training_gdf['type'] = training_gdf['type'].astype(int)

In [ ]:
# training_gdf['date']

In [ ]:
# training_gdf['year']

In [ ]:
import numpy as np
import rasterio
from affine import Affine
import rasterio.features

def get_raster_mask(date, ds, gdf):
    gdf_1 = gdf[gdf['type'] == 1]
    gdf_0 = gdf[gdf['type'] == 0]

    gdf_1 = gdf_1.dissolve(by='date')
    gdf_0 = gdf_0.dissolve(by='date')


    trees = gdf_1[gdf_1.index == date]
    non_trees = gdf_0[gdf_0.index == date]

    features = [(geom, 1) for geom in trees.geometry] + [(geom, 0) for geom in non_trees.geometry]

    # Assuming the dataset has evenly spaced coordinates named 'x' and 'y'
    x = ds.coords['x'].values
    y = ds.coords['y'].values

    # Calculate pixel resolution. 
    # Here we assume x is increasing and y is decreasing (common for image data)
    res_x = (x[-1] - x[0]) / (len(x) - 1)
    res_y = (y[0] - y[-1]) / (len(y) - 1)

    # Construct an affine transform.
    # The translation moves to the center of the top-left pixel.
    transform = Affine.translation(x[0] - res_x / 2, y[0] + res_y / 2) * Affine.scale(res_x, -res_y)

    mask_raster = rasterio.features.rasterize(
            features, 
            # [(geom, 1) for geom in gdf_1.geometry],
            out_shape=(len(y), len(x)),
            transform=transform,
            fill=np.nan,
            dtype="float32"
    )

    # Optionally wrap the raster in an xarray DataArray using ds coordinates
    mask_da = xr.DataArray(
        mask_raster,
        dims=("y", "x"),
        coords={"y": ds.coords["y"], "x": ds.coords["x"], "date": date},
    )

    return mask_da

In [ ]:
ds_resampled

In [ ]:
masks = []
for date in list(training_gdf.date.unique()):
    mask = get_raster_mask(date, ds_resampled, training_gdf)
    masks.append(mask)

In [ ]:
masks

In [ ]:
# merge mask_da_2021 and mask_da_2021 
mask_da = xr.concat(masks, dim='date')

# create mask that will containt True if all pixles in year range have valid value (ie. not nan)
mask = mask_da.notnull().all(dim='date')

In [ ]:
mask

In [ ]:
# convert to dataset 
gt = mask_da.to_dataset(name='ground_truth')
gt['valid'] = mask

In [ ]:
gt

In [ ]:
gt.date

In [ ]:
### INPUT
resampling_freq = 'MS'
# gt.date is already datetime, so just convert to pandas Timestamp
start_date = pd.Timestamp(gt.date.min().values)
cut_off_date = pd.Timestamp(gt.date.max().values)
print(f"Start date: {start_date}")
print(f"Cut-off date: {cut_off_date}")
cut_off_date

In [ ]:
# plot_gt_mask(gdf, gt, show=True, save=False)
# plot_gt_usable_data(gt, show=True, save=False)

# convert yeart in gt from int to datetime
# gt['date'] = pd.to_datetime(gt['date'].astype(str),  format='%Y%-m')

gt = gt.sortby('date').rename({'date': 'time'})
monthly_time = pd.date_range(start_date, cut_off_date, freq=resampling_freq)
# This will repeat the value from January 1 throughout the year until the next available time.
gt = gt.reindex(time=monthly_time, method='ffill').sortby('time')